# Strands Agents SDK と Claude 4 Interleaved Thinkingを活用したツールとしてのエージェント

このノートブックでは、Strands Agents SDK と Claude 4の**インターリーブ思考**機能を活用して、専門エージェントによるインテリジェントなワークフローを構築する方法を紹介します。

## インターリーブ思考を理解する

### インターリーブ思考とは？

インターリーブ思考は、Claude 4モデルの新機能で、モデルに以下の機能を提供します。

1. **ツール呼び出しの間に考える**：次のステップを決定する前に、結果を処理および推論する
2. **推論によって複数のツールを連鎖させる**：高度な複数ステップの意思決定を行う
3. **戦略を動的に適応させる**：中間結果に基づいてアプローチを変更する

### 仕組み

インターリーブ思考を適用したエージェントのイベントループと適用しないエージェントのイベントループには、多くの類似点があります。
```
クエリ → LLMが思考中 -> LLMがツールの呼び出しを決定 -> イベントループがツールを呼び出す -> 出力がLLMに返される -> [ LLMがツールを呼び出す必要がなくなるまでこれが続き、最終回答がレンダリングされます ]
```

インターリーブ思考で気付く主な違いは、イベントループがLLMの「決定」ではなく「思考」に基づいて動作する点です。上記のループの2番目のリンク、「thinking（思考）」に注目してください。従来のイベントループでは、思考は隠蔽されます。LLMがツールを呼び出す決定を下すか、最終回答を出すまで待たなければなりません。

インターリーブ思考の場合、LLMは2番目のステップ（「LLMは思考中」）にある間に、その思考をイベントループに「漏らし」ます。そして、イベントループはLLMがツールの実行を「考える」とすぐにツールを実行するように構成されています。つまり、LLMが思考を終える頃には、最初の「決定」で最終回答が実際に得られているということです。

### インターリーブ思考の有効化

Strands と Bedrock でこの機能を有効にするには、以下の手順に従います。
- `temperature=1` を設定します（思考が有効な場合は必須です）
- ベータヘッダーを追加します: `"anthropic_beta": ["interleaved-thinking-2025-05-14"]`
- 推論予算を設定します: `"reasoning_config": {"type": "enabled", "budget_tokens": 3000}`

## セットアップとインポート

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel
import os

## 専門エージェントをツールとして定義

まず、Strands `@tool` デコレータを使用して、4つの専門エージェントを作成します。
- **リサーチャー**: 事実情報を収集します
- **データアナリスト**: 情報を処理および分析します
- **ファクトチェッカー**: 情報の正確性を検証します
- **レポート作成者**: 洗練された最終文書を作成します

In [ ]:
# Strands @tool デコレータを使用してツールとして実装された専門エージェント
@tool
def researcher(query: str) -> str:
    """
    事実情報を収集する研究専門家。

    引数:
      クエリ: 調査する研究課題またはトピック

    戻り値:
      研究結果と情報源
    """
    # 集中的なリサーチエージェントを作成する
    # 注: 各呼び出しは新しいエージェントインスタンス（ステートレス）を作成します。
    research_agent = Agent(
        system_prompt="あなたは研究の専門家です。事実に基づいた情報を収集し、可能な場合は出典を明記してください。回答は200語以内に収めてください。",
        callback_handler=None  # ツールエージェントのストリーミングなし
    )
    
    # リサーチタスクを実行する
    result = research_agent(f"Research: {query}")
    return str(result)

In [ ]:
@tool
def data_analyst(data: str) -> str:
    """
    情報を処理・分析するデータアナリスト。

    引数:
      データ: 分析対象となる生データまたは調査結果

    戻り値:
      洞察とパターンに基づく分析
    """
    # アナリストエージェントは洞察の抽出に重点を置く
    analysis_agent = Agent(
        system_prompt="あなたはデータアナリストです。重要なインサイトを抽出し、パターンを特定し、分析的な結論を提示してください。実用的なインサイトに焦点を当ててください。",
        callback_handler=None
    )
    
    # 提供されたデータを分析
    result = analysis_agent(f"次のデータを分析して洞察を提供してください: {data}")
    return str(result)

In [ ]:
@tool
def fact_checker(information: str) -> str:
    """
    情報の正確性を検証するファクトチェッカー。

    引数:
      情報: 検証する主張またはデータ

    戻り値:
      正確性評価を含むファクトチェック結果
    """
    # 検証のためのファクトチェックエージェント
    fact_check_agent = Agent(
        system_prompt="あなたはファクトチェッカーです。主張を検証し、信憑性を評価し、信頼度を算出してください。疑わしい記述があれば特定してください。",
        callback_handler=None
    )
    
    # 情報を確認
    result = fact_check_agent(f"次の情報を事実確認します: {information}")
    return str(result)

In [ ]:
@tool
def report_writer(analysis: str) -> str:
    """
    洗練された最終文書を作成するレポートライター。

    引数:
      分析: 分析されたデータと洞察

    戻り値:
      フォーマットされた最終レポート
    """
    # プロフェッショナルな作品を生み出すライターエージェント
    writer_agent = Agent(
        system_prompt="あなたはプロのレポートライターです。エグゼクティブサマリーと実用的な推奨事項を盛り込んだ、明確で構造化されたレポートを作成して下さい。",
        callback_handler=None
    )
    
    # レポートを作成
    result = writer_agent(f"次に基づいてプロフェッショナルなレポートを作成します。 {analysis}")
    return str(result)

## インターリーブ思考を用いたClaude 4オーケストレーター

次に、インターリーブ思考を用いて専門エージェントをインテリジェントに連携させるClaude 4エージェントであるオーケストレーターを作成します。

### オーケストレーターの動作：

1. ユーザーから高レベルのタスクを受け取ります。
2. 必要な情報について**検討**します。
3. 初期データを収集するために研究者ツールを呼び出します。
4. 調査結果と必要な分析について**検討**します。
5. データアナリストを呼び出して調査結果を処理します。
6. 正確性と検証の必要性について**検討**します。
7. 必要に応じてファクトチェッカーを呼び出します。
8. 調査結果の提示方法について**検討**します。
9. 最終出力のためにレポート作成者を呼び出します。
10. 回答前にワークフロー全体を**検討**します。

In [ ]:
# クロード4 ストランドを用いたインターリーブ思考によるオーケストレーター
class StrandsInterlevedWorkflowOrchestrator:
    def __init__(self):
        # インテリジェントなワークフロー調整のためのオーケストレーターシステムプロンプトを定義する
        self.system_prompt = """あなたは、専門エージェントにアクセスできるインテリジェントなワークフローオーケストレーターです。
        あなたの役割は、これらの専門エージェントを活用して、ワークフローをインテリジェントに調整することです。
        - リサーチャー：あらゆるトピックに関する事実情報を収集します。
        - データアナリスト：データを分析し、洞察を抽出します。
        - ファクトチェッカー：情報の正確性を検証します。
        - レポートライター：洗練された最終レポートを作成します。
        """
    
    def run_workflow(self, task: str, enable_interleaved_thinking: bool = True) -> str:
        """指定されたタスクに対してインテリジェントなワークフローを実行します。

        引数:
          task: 完了するタスク
          enable_interleaved_thinking: インターリーブ思考を有効にするかどうか (デフォルト: True)

        オーケストレーターは以下の処理を行います。
        1. タスク要件を理解する
        2. 最適なアプローチを検討する
        3. 専門エージェントを調整する
        4. ステップ間の結果を反映する
        5. 包括的な出力を生成する
        """
        thinking_mode = "WITH interleaved thinking" if enable_interleaved_thinking else "WITHOUT interleaved thinking"
        print(f"\nStarting intelligent workflow {thinking_mode} for: {task}")
        print("=" * 70)
        
        # Bedrock 経由で Claude 4 をインターリーブ思考の有無で構成
        if enable_interleaved_thinking:
            claude4_model = BedrockModel(
                model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
                max_tokens=4096,
                temperature=1,  # 思考が有効な場合は1である必要がある
                additional_request_fields={
                    # インターリーブ思考ベータ機能を有効にする
                    "anthropic_beta": ["interleaved-thinking-2025-05-14"],
                    # 推論パラメータを設定
                    "reasoning_config": {
                        "type": "enabled",  # 思考をオンにする
                        "budget_tokens": 3000  # トークンバジェット
                    }
                }
            )
        else:
            claude4_model = BedrockModel(
                model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
                max_tokens=4096,
                temperature=1
            )
        
        # Claude 4と専門ツールを使ってオーケストレーターエージェントを作成する
        orchestrator = Agent(
            model=claude4_model,
            system_prompt=self.system_prompt,
            tools=[researcher, data_analyst, fact_checker, report_writer]
        )
        
        prompt = f"""インテリジェントなワークフロー調整を使用して、このタスクを完了してください: {task}

        手順:
        1. このタスクを達成するために必要な情報を慎重に検討してください。
        2. 専門エージェントを戦略的に活用してください。それぞれに独自の強みがあります。
        3. 各ツールの使用後、結果を振り返り、アプローチを調整してください。
        4. 包括的な結果を得るために、必要に応じて複数のエージェントを調整してください。
        5. 必要に応じて事実確認を行い、正確性を確保してください。
        6. すべての側面を網羅した包括的な最終回答を提供してください。

        覚えておいてください: ツール呼び出しの合間にじっくり考えることで、より良い意思決定が可能になります。
        計画、結果の評価、戦略の調整に活用してください。
        """
        
        try:
            result = orchestrator(prompt)
            return str(result)
        except Exception as e:
            return f"Workflow failed: {e}"

## デモを実行する

オーケストレーターの動作を見てみましょう！オーケストレーターがどのように考え、考えながらツールを呼び出すのかを見てみましょう。


In [ ]:
# オーケストレーターを作成する
print("Strands Agents SDK: Claude 4 Interleaved Thinking Workflow Demo")
print("=" * 70)

try:
    orchestrator = StrandsInterlevedWorkflowOrchestrator()
    print("✅ Orchestrator initialized successfully!")
except Exception as e:
    print(f"❌ Failed to initialize orchestrator: {e}")

In [ ]:
# テストケースでワークフローを実行
test_case = "リモートワークが生産性に与える影響を分析し、戦略的な推奨事項を提供します"

print(f"📋 Task: {test_case}\n")

try:
    result = orchestrator.run_workflow(test_case)
    
    print(f"\n📊 Workflow Result:")
    print("=" * 70)
    print(result)
except Exception as e:
    print(f"❌ Workflow execution failed: {e}")

## インターリーブ思考を無効にして試す

オーケストレーターを呼び出して、インターリーブ思考を無効にして実験してみましょう。出力の違いを観察します。

In [ ]:
# では、インターリーブ思考なしで同じタスクを試してみましょう
print("\n" + "="*70)
print("🔄 Now running the same task WITHOUT interleaved thinking")
print("="*70)

try:
    result_without_thinking = orchestrator.run_workflow(test_case, enable_interleaved_thinking=False)
    
    print(f"\n📊 Workflow Result (Without Interleaved Thinking):")
    print("=" * 70)
    print(result_without_thinking)
except Exception as e:
    print(f"❌ Workflow execution failed: {e}")